# 01: Exploring the API | RTEM Hackathon

## Import Packages
Let's start by importing the stuff we need.
This includes packages to work with JSON data, as well as the Onboard Client

In [2]:
# Import the packages we need to do our analysis
import json

import pandas as pd
from onboard.client import RtemClient

## Be Safe With Your Creds
Just because this is a Hackathon and a personal project doesn't mean we get to flippantly broadcast our API Keys to
the world. I want to publish this code to GitHub so let's be responsible and import our keys using configparser.

In [3]:
# Load API Key using configparser
import configparser

config = configparser.ConfigParser()
config.read('../config.ini')
api_key = config['DEFAULT']['API_KEY']

## Initialise the Onboard Client
Then check to see if it's working properly.

In [4]:
# Initialise Onboard Client
client = RtemClient(api_key=api_key)

# Check that everything is in order
whoami = client.whoami()
print(f"""
    Result: {whoami['result'].upper()}
    Scope: {whoami['apiKeyScopes']}
    Version: {whoami['apiVersion']}
    User: {whoami['userInfo']['full_name']}
    Org: {whoami['userInfo']['org_short_name']}
""")


    Result: OK
    Scope: ['buildings:read', 'general', 'auth']
    Version: 2022-04-14
    User: Daniel Lawson
    Org: RTEM Submission



## Get Information About the Buildings


In [20]:
# Create a DataFrame with all buildings
all_buildings = pd.json_normalize(client.get_all_buildings()).set_index('id')
all_buildings

,org_id,name,address,sq_ft,image_src,bms_manufacturer,bms_product_name,bms_version,timezone,status,...,info.sunend,info.geoCity,info.geoState,info.m2fstart,info.satstart,info.sunstart,info.yearBuilt,info.geoCountry,info.weatherRef,info.customerType
id,,,,,,,,,,,,,,,,,,,,,
441,5,88737,None,551143.0,None,None,None,None,America/New_York,LIVE,...,,Bronx,NY,,,,,US,,Multifamily
140,5,83483,None,21869.0,None,None,None,None,America/New_York,LIVE,...,,New York,NY,,,,,US,,Commercial Office
191,5,94414,None,198057.0,None,None,None,None,America/New_York,LIVE,...,23:00,Victor,NY,07:30,07:30,07:30,,US,,Commercial Retail
231,5,89496,None,NaN,None,None,None,None,America/New_York,LIVE,...,,New York,NY,,,,,US,,
248,5,116742,None,127000.0,None,None,None,None,America/New_York,LIVE,...,,New York,NY,,,,,US,,Multifamily
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,5,94413,None,198057.0,None,None,None,None,America/New_York,LIVE,...,23:00,Henrietta,NY,07:30,07:30,07:30,,US,,Commercial Retail
370,5,95457,None,421316.0,None,None,None,None,America/New_York,LIVE,...,,New York,NY,,,,,US,,Hospitality
383,5,79528,None,75000.0,None,None,None,None,America/New_York,LIVE,...,NaN,Purchase,New York,04:00:00,NaN,NaN,1971,US,NaN,Commercial Office


### What Kind of Buildings Are There?
There are several different types of buildings that participate in the RTEM Program.

In [6]:
# Get Building Types to DataFrame
building_types = all_buildings[['info.customerType']].rename(columns={'info.customerType': 'building_type'})
# Copy DataFrame to the Clipboard for README.md
pd.io.clipboards.to_clipboard(building_types.value_counts().to_markdown(), excel=False)
building_types.value_counts()

building_type     
Multifamily           69
Commercial Retail     49
Commercial Office     46
Healthcare            16
Food/Beverage         13
K-12 School            9
College/University     7
Hospitality            7
Not For Profit         5
                       3
Chemicals              1
Manufacturing          1
dtype: int64

In [7]:
all_buildings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 229 non-null    int64  
 1   org_id             229 non-null    int64  
 2   name               229 non-null    object 
 3   address            0 non-null      object 
 4   sq_ft              201 non-null    float64
 5   image_src          0 non-null      object 
 6   bms_manufacturer   0 non-null      object 
 7   bms_product_name   0 non-null      object 
 8   bms_version        0 non-null      object 
 9   timezone           229 non-null    object 
 10  status             229 non-null    object 
 11  equip_count        229 non-null    int64  
 12  point_count        229 non-null    int64  
 13  info.org           224 non-null    object 
 14  info.floors        224 non-null    object 
 15  info.m2fend        224 non-null    object 
 16  info.satend        223 non

In [19]:
all_units = pd.json_normalize(client.get_all_units()).set_index('id')
all_units

,name_long,name_abbr,data_type,raw_encoding,display_encoding,qudt,unit_type,raw_encoding.0,raw_encoding.1,display_encoding.0,display_encoding.1,raw_encoding.2,display_encoding.2
id,,,,,,,,,,,,,
23,Unknown,?,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN
21,Enable,Enable,Binary,NaN,NaN,None,None,[disable],[enable],Disable,Enable,NaN,NaN
22,Heat/Cool/Off,H/C/O,Enum,NaN,NaN,None,None,[off],[heat],Off,Heat,[cool],Off
31,Ordinal Value,Ordinal,Ordinal,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN
34,Ampere,A,Continuous,NaN,NaN,http://qudt.org/vocab/unit/A,http://qudt.org/vocab/quantitykind/ElectricCur...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Normal Cubic Meter,nm3,Continuous,NaN,NaN,http://qudt.org/vocab/unit/NM3-NAT-GAS,http://qudt.org/vocab/quantitykind/Energy,NaN,NaN,NaN,NaN,NaN,NaN
84,Standard Cubic Meter,SCM,Continuous,NaN,NaN,http://qudt.org/vocab/unit/SCM-NAT-GAS,http://qudt.org/vocab/quantitykind/Energy,NaN,NaN,NaN,NaN,NaN,NaN
85,Standard Cubic Foot,SFt3,Continuous,NaN,NaN,http://qudt.org/vocab/unit/SFT3-NAT-GAS,http://qudt.org/vocab/quantitykind/Energy,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
equip_types = pd.json_normalize(client.get_equipment_types())
equip_types[['id', 'name_long', 'name_abbr']].sort_values('name_long').set_index('id')

,id,name_long,name_abbr
36,49,Air Dryer,DRYER
0,12,Air Handling Unit,AHU
37,45,Battery,BATT
1,19,Boiler,BLR
2,20,Chilled Water Plant,CHWS
3,21,Chiller,CH
28,48,Cogeneration Plant,COGEN
20,47,Computer Room Air Conditioner,CRAC
4,22,Condenser,COND
35,44,Constant Air Volume,CAV


In [14]:
# Beware: this request takes a WHILE
all_equipment = pd.json_normalize(client.get_all_equipment())
all_equipment.set_index('id')

,id,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
0,28797,441,boiler-1,1,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[28803],[],"[{'id': 310029, 'building_id': 441, 'last_upda...","[boiler, hvac]"
1,28798,441,boiler-2,2,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[28803],[],"[{'id': 310079, 'building_id': 441, 'last_upda...","[boiler, hvac]"
2,28799,441,boiler-3,3,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[28803],[],"[{'id': 310108, 'building_id': 441, 'last_upda...","[boiler, hvac]"
3,28800,441,boiler-4,4,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[28803],[],"[{'id': 310144, 'building_id': 441, 'last_upda...","[boiler, hvac]"
4,28801,441,boiler-5,5,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[28803],[],"[{'id': 310152, 'building_id': 441, 'last_upda...","[boiler, hvac]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6080,32354,399,chilledWaterPump,None,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,None,None,[22751],[],"[{'id': 231126, 'building_id': 399, 'last_upda...","[pump, hvac, chilledWaterPump]"
6081,32355,399,chilledWaterPump-4,4,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,None,None,[22751],[],"[{'id': 359904, 'building_id': 399, 'last_upda...","[pump, hvac, chilledWaterPump]"
6082,32356,399,condenser,None,Condenser,22,COND,condenser,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 231136, 'building_id': 399, 'last_upda...","[hvac, condenser]"
6083,32357,399,condenserWaterPump,None,Pump,32,PUMP,pump,Condenser Water Pump,26.0,condenserWaterPump,1,None,None,None,None,[],[],"[{'id': 231132, 'building_id': 399, 'last_upda...","[pump, hvac, condenserWaterPump]"


In [15]:
all_equipment['equip_type_name'].value_counts()

Air Handling Unit            1165
Fan                           936
Meter                         801
Variable Air Volume           715
Pump                          510
Site Sensors                  370
Fan Coil Unit                 366
Unit Ventilator               193
Electrical Panel              153
Chiller                       151
Boiler                        140
Lighting System                76
Radiant System                 70
Cooling Tower                  65
Virtual                        63
Battery                        61
Hot Water Plant                59
Heat Exchanger                 57
Chilled Water Plant            50
Heat Pump                      34
Condenser                      18
Elevator                       16
Variable Refrigerant Flow      10
Duct Heater                     3
Tank                            2
Dry Cooler                      1
Name: equip_type_name, dtype: int64

In [16]:
all_measurements = pd.json_normalize(client.get_all_measurements())
all_measurements[['id', 'name']].set_index('id')

,name
id,
10,Occupancy
57,Dimensionless
11,On/Off
17,Motor Speed
31,Torque
7,Position
12,Enable
15,Open/Close
26,State


In [27]:
point_types = pd.json_normalize(client.get_all_point_types())
point_types\
    .join(all_measurements[['id', 'name']]\
    .set_index('id'), on='measurement_id')\
    .set_index('id')[['tag_name','name']]\
    .rename(columns={
            'name': 'measurement_type',
            'tag_name': 'point_type'
            })
point_types

,id,tag_name,active,measurement_id,tag_set_ids,tags,default_unit_id
0,124,Occupied Heating Setpoint,True,1,"[15, 2, 4, 62, 51, 55]","[air, sp, temp, zone, heating, occ]",None
1,118,Outside Air Carbon Dioxide,True,6,"[15, 8, 1, 24]","[air, co2, sensor, outside]",None
2,130,Return Air Temperature Setpoint,True,1,"[15, 2, 4, 63]","[air, sp, temp, return]",None
3,84,Dual-Temp Coil Discharge Air Temperature,True,1,"[15, 36, 47, 1, 4, 250]","[air, discharge, dualTemp, sensor, temp, coil]",None
4,81,Reheat Coil Discharge Air Temperature,True,1,"[15, 36, 44, 1, 4, 250]","[air, discharge, reheats, sensor, temp, coil]",None
...,...,...,...,...,...,...,...
642,749,Supply Water Pressure,True,3,"[1, 39, 6]","[sensor, leaving, pressure]",None
643,750,Return Water Pressure,True,3,"[1, 40, 6]","[sensor, entering, pressure]",None
644,752,Gas Flow Pulse,True,57,[],[],None
645,46,untagged,True,18,[],[],None


In [ ]:
all_points = pd.json_normalize(client.get_all_points()).set_index('id')
all_points

In [ ]:
all_points\
    .join(point_types[['id', 'tag_name', 'measurement_id']], on='point_type_id')